# Notebook to reproduce code as in model2roms carefully

In [2]:
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
import numpy as np
import time
import ESMF

In [3]:
# Import necessary data


# Reanalysis data
glorys_path = '/Users/iriskeizer/Documents/ROMS/data/GLORYS12V1/processed/glorys_test.nc'
ora_path = '/Users/iriskeizer/Documents/ROMS/data/ORA20C/forcing input/ora20_test.nc'

glorys_test = xr.open_dataset(glorys_path)
ora_test = xr.open_dataset(ora_path)



data_path = ora_path
data = xr.open_dataset(ora_path)

data_type = 'ORA20C' # Choose 'GLORYS' or 'ORA20C'


# Sort coordinates in increasing order
data = data.sortby(['depth', 'latitude', 'longitude'])



# ROMS grid
grid_path = '/Users/iriskeizer/Documents/ROMS/data/grid/NorthSea4_smooth01_sponge_nudg.nc'

grid = xr.open_dataset(grid_path)




# Tim files for comparison
ini_tim = xr.open_dataset('/Users/iriskeizer/Documents/ROMS/timnorthsea8/NS8_ForcingSmooth01/NS8_init_GLORYS_19930115_to_20021231.nc')
clim_tim = xr.open_dataset('/Users/iriskeizer/Documents/ROMS/timnorthsea8/NS8_ForcingSmooth01/NS8_clim_GLORYS_19930115_to_20021231.nc')
bry_tim = xr.open_dataset('/Users/iriskeizer/Documents/ROMS/timnorthsea8/NS8_ForcingSmooth01/NS8_bry_GLORYS_19930115_to_20021231.nc')

## Create grdMODEL and grdROMS (configM2R.py, grd.py, runM2R.py, model2roms.py)

In [13]:
def create_grdMODEL(data, grid, grd_filename):
    ''' Function to create the dataset grdMODEL '''
    
    grdMODEL = data.copy().drop(['thetao', 'so', 'uo', 'vo'])
    
    
    # Create variables
    
    grdMODEL['grdName'] = 'NorthSea4_test'
    grdMODEL['realm'] = 'ocean'
    
    
    grdMODEL['grdType'] = 'regular'
    grdMODEL['lonName'] = 'longitude'
    grdMODEL['latName'] = 'latitude'
    grdMODEL['depthName'] = 'depth'
    grdMODEL['fillval'] = -32767   # Change for ORA-20C, because is NaN
    
    grdMODEL['lon'] = data.longitude
    grdMODEL['lat'] = data.latitude
    grdMODEL['h'] = data.depth
    grdMODEL['nlevels'] = grdMODEL.h.size
    
    
    
    grdMODEL['hc'] = None

    grdMODEL['z_r'] = -grdMODEL.h

    

    grdMODEL['Lp'] = len(grid.xi_rho)
    grdMODEL['Mp'] = len(grid.eta_rho)

    grdMODEL['L'] = grdMODEL.Lp - 1
    grdMODEL['M'] = grdMODEL.Mp - 1
    

    
    # Create grid for ESMF interpolation
    
    
    grdMODEL['esmfgrid'] = ESMF.Grid(filename = grd_filename, filetype = ESMF.FileFormat.GRIDSPEC,
                                    is_sphere = True, coord_names = [data.longitude.name, data.latitude.name],
                                    add_mask = False)
    
    grdMODEL['esmfgrid_u'] = ESMF.Grid(filename = grd_filename, filetype = ESMF.FileFormat.GRIDSPEC,
                                    is_sphere = True, coord_names = [data.longitude.name, data.latitude.name],
                                    add_mask = False)
    
    grdMODEL['esmfgrid_v'] = ESMF.Grid(filename = grd_filename, filetype = ESMF.FileFormat.GRIDSPEC,
                                    is_sphere = True, coord_names = [data.longitude.name, data.latitude.name],
                                    add_mask = False)
    
    
    
    
    return grdMODEL

def create_grdROMS(grid, grd_filename):
    ''' Function to create the dataset grdROMS '''
    
    # Create the dataset grdROMS

    # Copy the roms grid
    grdROMS = grid.copy()

    # Drop unnecessary variables
    grdROMS = grdROMS.drop(['tracer_NudgeCoef', 'diff_factor', 'visc_factor', 'hraw', 'f', 'spherical'])

    # Add below variables to grdROMS
    
    grdROMS['nlevels'] = grdROMS.s_rho.size
    grdROMS['vtransform'] = 2
    grdROMS['vstretching'] = 4
    
    grdROMS['write_clim'] = True
    grdROMS['write_bry'] = True
    grdROMS['write_init'] = True
    grdROMS['write_stations'] = False
    grdROMS['lonname'] = 'lon_rho'
    grdROMS['latname'] = 'lat_rho'
    grdROMS['inittime'] = 0                    # Set initTime to 1 if you dont want the first time-step to be the initial field (no ubar and vbar if time=0)
    grdROMS['ocean_time'] = 0
    grdROMS['NT'] = 2
    grdROMS['tracer'] = grdROMS.NT
    grdROMS['time'] = 0                      
    grdROMS['reftime'] = 0
    grdROMS['grdtype'] = 'regular'

    grdROMS['masked_h'] = grdROMS.h.where(grdROMS.h > 0, grdROMS.h, grdROMS.h.max())
    grdROMS['hmin'] = grdROMS.masked_h.min()

    

    grdROMS['zeta'] = (('eta_rho', 'xi_rho'), np.zeros(grdROMS.h.shape))

    grdROMS['invpm'] = 1.0 / grdROMS.pm
    grdROMS['invpn'] = 1.0 / grdROMS.pn

    grdROMS['Lp'] = grdROMS.lat_rho[1,:].size     
    grdROMS['Mp'] = grdROMS.lat_rho[:,1].size     

    grdROMS['fillval'] = -9.99e33

    grdROMS['eta_rho_'] = grdROMS.Mp
    grdROMS['eta_u_'] = grdROMS.Mp
    grdROMS['eta_v_'] = grdROMS.Mp - 1
    grdROMS['eta_psi_'] = grdROMS.Mp - 1


    grdROMS['xi_rho_'] = grdROMS.Lp
    grdROMS['xi_u_'] = grdROMS.Lp - 1
    grdROMS['xi_v_'] = grdROMS.Lp
    grdROMS['xi_psi_'] = grdROMS.Lp - 1



    # Obtain s_rho

    c1 = 1.0
    c2 = 2.0
    p5 = 0.5

    lev = np.arange(1, int(grdROMS.nlevels) + 1, 1)
    ds = 1.0 / int(grdROMS.nlevels)


    grdROMS['s_rho_'] = - c1 + (lev - p5) * ds


    # Obtain s_w

    lev = np.arange(0, int(grdROMS.nlevels), 1)
    ds = 1.0 / (int(grdROMS.nlevels) - 1)


    grdROMS['s_w_'] = - c1 + (lev - p5) * ds




    # Obtain Cs_r

    if (grdROMS.theta_s > 0):
        Csur = (c1 - np.cosh(grdROMS.theta_s * grdROMS.s_rho)) / (np.cosh(grdROMS.theta_s) - c1)

    else:
        Csur = -grdROMS.s_rho**2

    if (grdROMS.theta_b > 0):
        Cbot = (np.exp(grdROMS.theta_b * Csur) - c1 ) / (c1 - np.exp(-grdROMS.theta_b))
        grdROMS['Cs_r'] = Cbot
    else:
        grdROMS['Cs_r'] = Csur     



    # Obtain Cs_w

    if (grdROMS.theta_s > 0):
        Csur = (c1 - np.cosh(grdROMS.theta_s * grdROMS.s_w)) / (np.cosh(grdROMS.theta_s) - c1)

    else:
        Csur = -grdROMS.s_w**2

    if (grdROMS.theta_b > 0):
        Cbot = (np.exp(grdROMS.theta_b * Csur) - c1 ) / (c1 - np.exp(-grdROMS.theta_b))
        grdROMS['Cs_w'] = Cbot
    else:
        grdROMS['Cs_w'] = Csur     




    # Obtain z_r

    z0 = (grdROMS.hc * grdROMS.s_rho + grdROMS.h * grdROMS.Cs_r) / (grdROMS.hc + grdROMS.h)
    grdROMS['z_r'] = grdROMS.zeta + (grdROMS.zeta + grdROMS.h) * z0



    # Obtain z_w

    z0 = (grdROMS.hc * grdROMS.s_w + grdROMS.h * grdROMS.Cs_w) / (grdROMS.hc + grdROMS.h)
    grdROMS['z_w'] = grdROMS.zeta + (grdROMS.zeta + grdROMS.h) * z0



    

    grdROMS['L'] = grdROMS.Lp -1
    grdROMS['M'] = grdROMS.Mp -1

    
    
    # Create grid for ESMF interpolation
    
    
    grdROMS['esmfgrid'] = ESMF.Grid(filename = grd_filename, filetype = ESMF.FileFormat.GRIDSPEC,
                                    is_sphere = True, coord_names = ['lon_rho', 'lat_rho'],
                                    add_mask = False)
    
    grdROMS['esmfgrid_u'] = ESMF.Grid(filename = grd_filename, filetype = ESMF.FileFormat.GRIDSPEC,
                                    is_sphere = True, coord_names = ['lon_u', 'lat_u'],
                                    add_mask = False)
    
    grdROMS['esmfgrid_v'] = ESMF.Grid(filename = grd_filename, filetype = ESMF.FileFormat.GRIDSPEC,
                                    is_sphere = True, coord_names = ['lon_v', 'lat_v'],
                                    add_mask = False)
    
    
    
    
    
    return grdROMS



grdMODEL = create_grdMODEL(data, grid, data_path)
grdROMS = create_grdROMS(grid, grid_path)

## Create the ESMF weights used to do all of the horizontal interpolations (model2roms.py)


interp2D.setup_ESMF_interpolation_weights(confM2R)

In [28]:
grdROMS.esmfgrid

<xarray.DataArray 'esmfgrid' ()>
array(Grid:
    type = <TypeKind.R8: 6>    areatype = <TypeKind.R8: 6>    rank = 2    num_peri_dims = 1    periodic_dim = 0    pole_dim = None    pole_kind = None    coord_sys = None    staggerloc = [True, False, False, False]    lower bounds = [array([0, 0], dtype=int32), None, None, None]    upper bounds = [array([122, 110], dtype=int32), None, None, None]    coords = [[array([[-19.87704918, -19.87704918, -19.87704918, ..., -19.87704918,
        -19.87704918, -19.87704918],
       [-19.63114754, -19.63114754, -19.63114754, ..., -19.63114754,
        -19.63114754, -19.63114754],
       [-19.3852459 , -19.3852459 , -19.3852459 , ..., -19.3852459 ,
        -19.3852459 , -19.3852459 ],
       ...,
       [  9.3852459 ,   9.3852459 ,   9.3852459 , ...,   9.3852459 ,
          9.3852459 ,   9.3852459 ],
       [  9.63114754,   9.63114754,   9.63114754, ...,   9.63114754,
          9.63114754,   9.63114754],
       [  9.87704918,   9.87704918,   9.87704918, ...,   9.87704918,
          9.87704918,   9.87704918]]), array([[35.95173306, 36.25432606, 36.55575166, ..., 61.55725605,
        61.73511889, 61.91196069],
       [35.95173306, 36.25432606, 36.55575166, ..., 61.55725605,
        61.73511889, 61.91196069],
       [35.95173306, 36.25432606, 36.55575166, ..., 61.55725605,
        61.73511889, 61.91196069],
       ...,
       [35.95173306, 36.25432606, 36.55575166, ..., 61.55725605,
        61.73511889, 61.91196069],
       [35.95173306, 36.25432606, 36.55575166, ..., 61.55725605,
        61.73511889, 61.91196069],
       [35.95173306, 36.25432606, 36.55575166, ..., 61.55725605,
        61.73511889, 61.91196069]])], [None, None], [None, None], [None, None]]    mask = [None, None, None, None]    area = [None, None, None, None],
      dtype=object)

In [30]:
def setup_ESMF_interpolation_weights(grdROMS, grdMODEL):
    '''
    Function to create the ESMF weights for horizontal interpolations
    '''
    
    
    # Source field src at RHO points
    grdMODEL.fieldSrc_rho = ESMF.Field(grdMODEL.esmfgrid, "fieldSrc",
                                      staggerloc = ESMF.StaggerLoc)
    
    
    # Destination field src at RHO, u, and v points
    grdROMS.fieldDst_rho = ESMF.Field(grdROMS.esmfgrid, "fieldDst",
                                   staggerloc = ESMF.StaggerLoc.CENTER)
    
    grdROMS.fieldDst_u = ESMF.Field(grdROMS.esmfgrid_u, "fieldDst",
                                   staggerloc = ESMF.StaggerLoc.CENTER)
    
    grdROMS.fieldDst_v = ESMF.Field(grdROMS.esmfgrid_v, "fieldDst",
                                   staggerloc = ESMF.StaggerLoc.CENTER)
    
    
    # RegridSrc2Dst from RHO to U, V and RHO points
    grdROMS.regridSrc2Dst_rho = ESMF.Regrid(grdMODEL.fieldSrc_rho,
                                           grdROMS.fieldDst_rho,
                                           regrid_method = ESMF.RegridMethod.BILINEAR,
                                           unmapped_action = ESMF.UnmappedAction.IGNORE)
    
    grdROMS.regridSrc2Dst_u = ESMF.Regrid(grdMODEL.fieldSrc_rho,
                                           grdROMS.fieldDst_u,
                                           regrid_method = ESMF.RegridMethod.BILINEAR,
                                           unmapped_action = ESMF.UnmappedAction.IGNORE)
    
    grdROMS.regridSrc2Dst_v = ESMF.Regrid(grdMODEL.fieldSrc_rho,
                                           grdROMS.fieldDst_v,
                                           regrid_method = ESMF.RegridMethod.BILINEAR,
                                           unmapped_action = ESMF.UnmappedAction.IGNORE)
    
    
    return grdMODEL, grdROMS


grdMODEL, grdROMS = setup_ESMF_interpolation_weights(grdROMS, grdMODEL)

FieldDOError: 

In [17]:
grdMODEL.esmfgrid

<xarray.DataArray 'esmfgrid' ()>
array(Grid:
    type = <TypeKind.R8: 6>    areatype = <TypeKind.R8: 6>    rank = 2    num_peri_dims = 1    periodic_dim = 0    pole_dim = None    pole_kind = None    coord_sys = None    staggerloc = [True, False, False, False]    lower bounds = [array([0, 0], dtype=int32), None, None, None]    upper bounds = [array([38, 34], dtype=int32), None, None, None]    coords = [[array([[ 0.5,  0.5,  0.5, ...,  0.5,  0.5,  0.5],
       [ 1.5,  1.5,  1.5, ...,  1.5,  1.5,  1.5],
       [ 2.5,  2.5,  2.5, ...,  2.5,  2.5,  2.5],
       ...,
       [-2.5, -2.5, -2.5, ..., -2.5, -2.5, -2.5],
       [-1.5, -1.5, -1.5, ..., -1.5, -1.5, -1.5],
       [-0.5, -0.5, -0.5, ..., -0.5, -0.5, -0.5]]), array([[32.5, 33.5, 34.5, ..., 63.5, 64.5, 65.5],
       [32.5, 33.5, 34.5, ..., 63.5, 64.5, 65.5],
       [32.5, 33.5, 34.5, ..., 63.5, 64.5, 65.5],
       ...,
       [32.5, 33.5, 34.5, ..., 63.5, 64.5, 65.5],
       [32.5, 33.5, 34.5, ..., 63.5, 64.5, 65.5],
       [32.5, 33.5, 34.5, ..., 63.5, 64.5, 65.5]])], [None, None], [None, None], [None, None]]    mask = [None, None, None, None]    area = [None, None, None, None],
      dtype=object)

$\eta$ $\xi$

In [11]:
grid

<xarray.Dataset>
Dimensions:           (s_rho: 30, s_w: 31, eta_rho: 110, xi_rho: 122, bath: 1,
                       eta_u: 110, xi_u: 121, eta_v: 109, xi_v: 122,
                       eta_psi: 109, xi_psi: 121, eta_vert: 111, xi_vert: 123)
Coordinates:
  * s_rho             (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
  * s_w               (s_w) float64 -1.0 -0.9667 -0.9333 ... -0.03333 0.0
Dimensions without coordinates: eta_rho, xi_rho, bath, eta_u, xi_u, eta_v,
                                xi_v, eta_psi, xi_psi, eta_vert, xi_vert
Data variables: (12/44)
    theta_s           float64 ...
    theta_b           float64 ...
    Tcline            float64 ...
    hc                float64 ...
    Cs_r              (s_rho) float64 ...
    Cs_w              (s_w) float64 ...
    ...                ...
    mask_u            (eta_u, xi_u) float64 ...
    mask_v            (eta_v, xi_v) float64 ...
    mask_psi          (eta_psi, xi_psi) float64 ...
    visc_factor       (eta_rho, xi_rho) float64 ...
    diff_factor       (eta_rho, xi_rho) float64 ...
    tracer_NudgeCoef  (s_rho, eta_rho, xi_rho) float64 ...
Attributes:
    Description:  ROMS grid
    Author:       pyroms.grid.write_grd
    Created:      2016-05-13T10:47:07.357379
    type:         ROMS grid file

In [5]:
ora_test

<xarray.Dataset>
Dimensions:    (time: 60, depth: 42, latitude: 34, longitude: 38)
Coordinates:
  * depth      (depth) float32 5.022 15.08 25.16 ... 5.051e+03 5.35e+03
  * time       (time) datetime64[ns] 1950-01-16T12:00:00 ... 1954-12-16T12:00:00
  * longitude  (longitude) float64 0.5 1.5 2.5 3.5 4.5 ... -3.5 -2.5 -1.5 -0.5
  * latitude   (latitude) float32 32.5 33.5 34.5 35.5 ... 62.5 63.5 64.5 65.5
Data variables:
    so         (time, depth, latitude, longitude) float32 ...
    thetao     (time, depth, latitude, longitude) float32 ...
    uo         (time, depth, latitude, longitude) float32 ...
    vo         (time, depth, latitude, longitude) float32 ...
    zos        (time, latitude, longitude) float32 ...
Attributes:
    _CoordinateAxes:    time depth lat lon
    associate:          time depth lat lon
    interval_write:     monthly
    long_name:          Salinity
    offline_operation:  time average and spatial interpolation
    standard_name:      Salinity
    units:              psu

In [8]:
len(data.latitude)

34